# Tokenization & Language Models
We will study **tokenization** and **language modeling**. In particular, WordPiece tokenizer and some n-gram language models on a corpus of Wikipedia articles.

In [ ]:
!pip install datasets

# Part 1: WordPiece Tokenization [28 points]

## Download & preprocess the data

We use a subset of the FineWeb BBC News dataset. You can see the Huggingface dataset card [here](https://huggingface.co/datasets/permutans/fineweb-bbc-news) (including a discussion of potential limitations or biases), and the corresponding research paper [here](https://arxiv.org/pdf/2406.17557).

In [ ]:
import os
import pandas as pd
from datasets import load_dataset
import random
from tqdm.notebook import tqdm

class NewsDataset():
    """
    A class to manage and preprocess the News dataset for this homework.

    Attributes:
        train (list[str]): The training dataset as a list of sentences.
        test (list[str]): The testing dataset as a list of sentences.
    """
    def __init__(self, redownload_dataset=False):
        """
        Initializes the NewsDataset object. If the dataset files do not exist
        or redownload_dataset is set to True, it downloads the dataset.

        Args:
            redownload_dataset (bool): If True, redownloads the dataset.
        """
        self.train, self.test = [], []
        self.load_data(redownload_dataset)

    def __str__(self):
        """
        Returns a string representation of the dataset, including the number of
        training and testing examples.

        Returns:
            str: Information about the dataset size
        """
        repr = {}
        repr["train"] = len(self.train) if self.train else 0
        repr["test"] = len(self.test) if self.test else 0
        return f"{type(self).__name__}({repr})"

    @staticmethod
    def isValid(s: str) -> bool:
        """
        Checks whether a given string is valid for inclusion in the dataset.
        A string is considered valid if it contains more than five words and
        at least one alphabetic character.

        Args:
            s (str): The string to validate.

        Returns:
            bool: True if the string is valid, False otherwise.
        """
        return s and (len(s.strip().split()) > 5) and any(c.isalpha() for c in s)

    @staticmethod
    def preprocess(paragraph: str):
        paragraph = paragraph.translate(str.maketrans({'?': '.', '!': '.'}))
        sentences = paragraph.split('.')
        sentences = [s.replace("\"", "").replace("\'", "").strip().replace("\n", " ") + '.' for s in sentences]
        sentences = [s.lower() for s in sentences if '  ' not in s]
        sentences = [s for s in sentences if not any(char.isdigit() for char in s)]
        return sentences

    def preprocessWP(self, data):
        """
        Preprocesses a dataset by tokenizing them into
        sentences and filtering out invalid ones.

        Args:
            data: An iterable containing the text of articles.

        Returns:
            list[str]: A list of valid sentences.
        """
        dataset = []
        for item in data:
            sentences = self.preprocess(item["text"])
            sentences = [sent for sent in sentences if self.isValid(sent)]
            dataset += sentences
        return dataset

    def downloadDatasetWP(self):
        """
        Downloads and preprocesses the News dataset.

        Returns:
            list[str]: A list of preprocessed sentences from the dataset.
        """
        ds = load_dataset("permutans/fineweb-bbc-news", "sample-10BT")
        ds = self.preprocessWP(ds["train"])
        return ds

    def generateDatasetWP(self):
        """
        Downloads, preprocesses, and splits the dataset into training and
        testing sets.
        The processed data is saved as parquet files.
        """
        ds = self.downloadDatasetWP()
        data = {"text": ds}
        df = pd.DataFrame(data)
        train = df.sample(frac=0.05,random_state=200)
        test = train.sample(frac=0.1,random_state=200)
        train = train.drop(test.index)
        train.to_parquet('train.parquet', index=False)
        test.to_parquet('test.parquet', index=False)

    def load_data(self, redownload_dataset):
        """
        Loads the dataset from parquet files. If the files do not exist or
        redownload_dataset is set to True, it downloads the dataset.
        """
        if (redownload_dataset or
                not os.path.exists('train.parquet') or
                not os.path.exists('test.parquet')):
            self.generateDatasetWP()
        self.train, self.test = (pd.read_parquet('train.parquet')["text"].tolist(),
                                 pd.read_parquet('test.parquet')["text"].tolist())


if __name__ == '__main__':
    dataset = NewsDataset()

    print('\n', dataset, '\n\nExample sentences:')

    for sentence in dataset.train[:10]:
        print('\t', sentence)

## Tokenizer

First, we will write a training (learning) algorithm, which takes in a corpus (list of sentences), and returns a token vocabulary. Then, we will write the tokenization algorithm. This takes in a sentence and returns its tokenization based on the vocabulary.

Code for this section will all be in the `WordPieceTokenizer` class, where you will implement several functions.

### Training algorithm

WordPiece learns a token vocabulary in an iterative fashion. Starting with an initial token vocabulary (the set of characters in the training data), each step merges a pair of consecutive tokens, where the pair of tokens is selected according to a scoring function (described below). The process is continued until a desired `vocab_size` is reached.

1. **`initialize(self, train_corpus)`: [6 points]** This returns the initial vocabulary, the initial tokenization of each word, the word frequncies, and a mapping from each token to the words containing that token. First, split each sentence in `train_corpus` on the space character. The initial vocabulary is the set of all characters that occur in any word; but characters occuring in the middle of a word should be prepended with the special sequence `##`. For example, if your corpus is the sentence `the old man the boat.`, your initial vocabulary would be:
<div align="center"><code>{'t', '##h', '##e', 'o', '##l', '##d', 'm', '##a', '##n', 'b', '##o', '##t', '##.'}</code></div>
This function should also compute the initial tokenization of each word, storing them in a dictionary from word to tokenization. For example:
<div align="center"><code>{'the': ('t', '##h', '##e;), 'old': ('o', '##l', '##d'),                
    'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'}
</code></div>
The function should also count the frequency of each word in the training corpus, and return a mapping from token to the words that use that token. For example:
<div align="center"><code>word_freqs = {'the': 100, 'old': 34, 'man': 76, 'boat.': 18}</code></div>
    <div align="center"><code>tokens2word = {'t': {'the'}, '##h': {'the'}, '##e': {'the'}, ...}</code></div>

2. **`find_best_pair(self, word_tokenizations, word_freqs, vocab)`: [6 points]** This function computes a score for each *consecutive pair* $(t_1, t_2)$ of tokens, and returns the pair of tokens with highest score. The scoring function is $$\frac{c(t_1, t_2) \cdot |V|}{c(t_1)\cdot c(t_2)}$$where $c(t_1, t_2)$ is the number of times $t_1$ and $t_2$ occur consecutively in the corpus, $c(t_i)$ is the nubmer of times token $t_i$ occurs, and $|V|$ is the size of the current token vocabulary. The function will return the pair with highest score (ties broken alphabetically).

    The idea behind the scoring function is that a pair of tokens will have high score if they occur frequently together *and* each token does not occur frequently on its own. Factoring in the size of the vocabulary gives a (slight) preference to longer subword units as the vocabulary size grows.
    

3. **`merge_best_pair(self, best_pair, vocab, word_tokenizations, tokens2word)`: [6 points]** This function updates your current vocabulary (`vocab`), tokenizations (`word_tokenizations`), and token-to-word mapping (`tokens2word`) based on the pair of tokens to merge (`best_pair`). The `vocab` should simply be updated to include the new token, and any word in `word_tokenizations` that contains the consecutive pair of tokens should be updated to merge that pair. For example, suppose the pair of tokens to merge is `(##h, ##e)` and `word_tokenizations` is
<div align="center"><code>  {'the': ('t', '##h', '##e;), 'old': ('o', '##l', '##d'),                
    'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'}</code></div>
Then <code>word_tokenizations</code> should be updated to
<div align="center"><code>  {'the': ('t', '##he'), 'old': ('o', '##l', '##d'),                      
    'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'}</code></div>
Similarly, <code>tokens2word</code> should be updated to include the new token:
<div align="center"><code>tokens2word['##he'] = {'the'}</code></div>


### Tokenization algorithm

Finally, you will write the `tokenize` and `detokenize` functions, which use the token vocabulary you learned to tokenize and detokenize arbitrary sentences:

1. **`tokenize(self, sentence)`: [5 points]** This function takes a sentence and returns its tokenization. First, split on the space character. For each word, find the longest substring starting at the beginning of the word that is in the token vocabulary, and choose that token. Then, iteratively repeat this procedure on the remainder of the word, until the entire word is tokenized. For example, consider the word `swiftly`:
    * If `s` and `sw` are tokens but `swi` is not, then choose `sw`.
    * Then, proceed with `##iftly`. If `##i`, `##if`, and `##ift` are tokens, but `##iftl` is not, choose `##ift`.
    * Then, proceed with `##ly`. If both `##l` and `##ly` are tokens, choose `##ly`.

  After tokenizing each word in this way, return a list containing the tokenization of each word separated by space tokens. For the sentence `he is swift.`, for example, you might return `['he', ' ', 'is', ' ', 'sw', '##ift', '##.']`

  **Note:** If at any step during the tokenization of a word, you cannot find a matching token in your vocabulary, then the *entire* word should be tokenized as `<UNK>`. For example, if you are considering `##mat` but `##m`, `##ma`, and `##mat` are not in the vocabulary, the entire word (including the part of the word preceeding `##mat`) should be tokenized as `<UNK>`.


2. **`detokenize(self, tokens)`: [5 points]** This takes a list of tokens and returns its corresponding sentence. Simply copy tokens into a string from left to right, being careful to remove `##` at the beginning of tokens.

In [ ]:
class WordPieceTokenizer(object):
    def __init__(self, train_corpus, vocab_size, do_print=False, do_tqdm=True):
        '''

        Trains a WordPiece tokenizer.
        Args:
            train_corpus: List of strings (each string is a sentence)
            vocab_size: Integer indicating how big the vocabulary should be
            do_print: Whether or not to print merges
        '''
        self.train_corpus = train_corpus
        self.vocab_size = vocab_size
        self.do_print = do_print
        self.do_tqdm = do_tqdm
        for sent in train_corpus: assert '##' not in sent, sent
        self.vocab = None

    def initialize(self, train_corpus):
        '''
        Initialize token vocabulary, word frequencies, and token-to-word dictionaries.
        Args:
            train_corpus: List of strings, where each string is a sentence
        Returns:
            vocab: A set containing the initial token vocabulary (each token is a string)
            word_tokenizations: A dictionary of word to its tokenization (a tuple of strings)
            word_freqs: A dictionary that maps each word to its frequency in the training data
            tokens2word: A dictionary that maps each token to the set of words with that token
        '''

        vocab = {'<UNK>', ' '} # Initialize vocab with special tokens (don't change this)
        word_tokenizations, word_freqs, tokens2word = {}, {}, {}

        ### TODO ###

        return vocab, word_tokenizations, word_freqs, tokens2word

    def find_best_pair(self, word_tokenizations, word_freqs, vocab):
        '''
        Score all pairs of consecutive tokens (bigrams) in train_corpus, and return the pair with
          highest score. If there is a tie, choose the pair that is first alphabetically.
        Args:
            word_freqs: Dictionary of word to frqeuency
            word_tokenizations: Dictionary of word to its tokenization (a tuple of strings)
            vocab: A set of tokens (strings)
        Returns:
            best_pair: The pair (tuple) of tokens (t1, t2) that has highest score
            scores: Dictionary of tokens (t1, t2) to its score
        '''

        best_pair, scores = None, {}

        ### TODO ###

        return best_pair, scores

    def merge_best_pair(self, best_pair, vocab, word_tokenizations, tokens2word):
        '''
        Update vocab, word_tokenizations, and tokens2word based on pair to merge.
        Args:
            best_pair: The pair of tokens that had highest score; that is, the pair to be merged
            vocab: The token vocabulary (set of strings)
            word_tokenizations: Dictionary from word to tokenization
            tokens2word: A dictionary from token to the set of words with that token
        Returns: Nothing
        Modifies:
            vocab: The new token should be added to the vocab set.
            word_tokenizations: Any word that contains the consecutive pair of tokens in best_pair
              should be re-tokenized such that that pair of tokens has been merged.
            tokens2word: The new token should be mapped to the set of words that use this token;
              do NOT otherwise modify this dictionary.
        Hint: When looking for words that contain the new token, look ONLY at words that use either of
          the tokens in best_pair (by using tokens2word).
        '''

        ### TODO ###

        pass


    def train(self):
        '''

        Trains the WordPiece tokenization algorithm by calling iteratively merging tokens until
          vocab_size is reached.
        Args:
            train_corpus: List of strings, where each string is a sentence
            vocab_size: The desired vocabulary size
        Returns:
            vocab: The set of tokens in the vocabulary, returned as a sorted list
        '''
        vocab, word_tokenizations, word_freqs, tokens2word = self.initialize(self.train_corpus)
        init_vocab_size = len(vocab)
        if self.do_print: print("Initial vocab size:", init_vocab_size)
        assert self.vocab_size >= len(vocab), 'Cannot give a vocab size smaller than initial vocab size'

        itr = tqdm(range(self.vocab_size - len(vocab))) if self.do_tqdm else range(self.vocab_size - len(vocab))
        for i in itr:
            best_pair, scores = self.find_best_pair(word_tokenizations, word_freqs, vocab)
            self.merge_best_pair(best_pair, vocab, word_tokenizations, tokens2word)
            outputs = (vocab, word_tokenizations, tokens2word)
            # print("\tMerging ", best_pair) # UNCOMMENT IF YOU WANT TO SEE THE MERGES
            assert len(vocab) == init_vocab_size + i + 1, str(len(vocab)) + ' '+ str(init_vocab_size) + ' ' + str(i)
            if all(len(word_tokenizations[x]) == 1 for x in word_tokenizations):
                print("All words have been maximally merged at vocab_size=" +str(len(vocab)) + " – breaking.")
                break
        if self.do_print: print("Done!")
        self.vocab = sorted(vocab)


    def tokenize(self, sentence):
        '''
        Tokenizes a sentence.
        Args:
            sentence: A string representing a sentence
        Returns:
            tokens: A list containing the tokens of the sentence
        '''
        assert type(sentence) == str
        tokens = []

        ### TODO ###

        return tokens

    def detokenize(self, tokens):
        '''
        Detokenizes a sentence.
        Args:
            tokens: A list of tokens representing a sentence
        Returns:
            sentence: A string representing the sentence
        '''
        assert type(tokens) == list and len(tokens) > 0 and type(tokens[0]) == str
        sentence = ''


        return sentence

##Sanity Check: Tokenizer Class

The code below runs a sanity check for `WordPieceTokenizer` class. 

In [ ]:
# @title Sanity Check Code

def check_dictionary(test_dict, correct_dict):
    if not isinstance(test_dict, dict) or not isinstance(correct_dict, dict):
        return "INCORRECT", "Is not a dictionary."
    if set(test_dict.keys()) != set(correct_dict.keys()):
        return "INCORRECT", f"Key mismatch: Missing {set(correct_dict.keys()) - set(test_dict.keys())}, Extra {set(test_dict.keys()) - set(correct_dict.keys())}"
    for key in correct_dict:
        if test_dict[key] != correct_dict[key]:
            return "INCORRECT", f"Value mismatch for key '{key}': Expected {correct_dict[key]}, Got {test_dict[key]}"
    return "CORRECT", ''

def check_set(test_set, correct_set):
    if not isinstance(test_set, set) or not isinstance(correct_set, set):
        return "INCORRECT", "Is not a set."
    if test_set != correct_set:
        return "INCORRECT", f"Set mismatch: Missing {correct_set - test_set}, Extra {test_set - correct_set}"
    return "CORRECT", ""

def sanityCheckInitialize(train):
    tokenizer = WordPieceTokenizer(train, 45, do_print=False, do_tqdm=False)

    print('\n\n--- TEST: initialize(self, train_corpus) ---')
    res = tokenizer.initialize(train)
    if len(res) != 4:
        print('FAILED\ninitialize(self, train_corpus) must return 4 items.')
        return
    vocab, word_tokenizations, word_freqs, tokens2word = res

    correct_vocab = {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}
    correct_word_tokenizations = {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##o', '##w', '##;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}
    correct_word_freqs = {'the': 2, 'old': 1, 'man': 1, 'boat.': 1, 'time': 1, 'flies': 2, 'like': 2, 'an': 1, 'arrow;': 1, 'fruit': 1, 'a': 1, 'banana.': 1, 'she': 2, 'told': 1, 'him': 1, 'that': 2, 'knew': 1, 'he': 1, 'lied.': 1}
    correct_tokens2word = {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'lied.', 'old', 'told'}, 'm': {'man'}, '##a': {'that', 'banana.', 'boat.', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'that', 'boat.', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'lied.', 'like'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}}

    pass1, msg1 = check_set(vocab, correct_vocab)
    pass2, msg2 = check_dictionary(word_tokenizations, correct_word_tokenizations)
    pass3, msg3 = check_dictionary(word_freqs, correct_word_freqs)
    pass4, msg4 = check_dictionary(tokens2word, correct_tokens2word)

    print('\tvocab:\t\t\t'+pass1+'\t'+msg1 + '\tYour vocab: ' +str(vocab) + '\tCorrect vocab:' + str(correct_vocab))
    print('\tword_tokenizations:\t'+pass2+'\t'+msg2 + '\tYour word_tokenizations: ' +str(word_tokenizations) + '\tCorrect word_tokenizations: ' + str(correct_word_tokenizations))
    print('\tword_freqs:\t\t'+pass3+'\t'+msg3 + '\tYour word_freqs: ' +str(word_freqs) + '\tCorrect word_freqs: ' + str(correct_word_freqs))
    print('\ttokens2word:\t\t'+pass4+'\t'+msg4 + '\tYour tokens2word: ' +str(tokens2word) + '\tCorrect tokens2word: ' + str(correct_tokens2word))

    if len({pass1, pass2, pass3, pass4}) == 1 and pass1 == 'CORRECT':
        print('\n  Passed!')
    else: print('  Failed.')

def sanityCheckFindBestPair(train):
    print('\n\n--- TEST: find_best_pair(self, word_tokenizations, word_freqs, vocab) ---') # max_len does not matter for this test
    test_cases = ({'the': 2, 'old': 1, 'man': 1, 'boat.': 1, 'time': 1, 'flies': 2, 'like': 2, 'an': 1, 'arrow;': 1, 'fruit': 1, 'a': 1, 'banana.': 1, 'she': 2, 'told': 1, 'him': 1, 'that': 2, 'knew': 1, 'he': 1, 'lied.': 1}, [(({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##o', '##w', '##;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}), (('##w', '##;'), {('t', '##h'): 3.2222222222222223, ('##h', '##e'): 1.6111111111111112, ('o', '##l'): 7.25, ('##l', '##d'): 4.833333333333333, ('m', '##a'): 4.142857142857143, ('##a', '##n'): 2.4857142857142858, ('b', '##o'): 4.833333333333333, ('##o', '##a'): 1.380952380952381, ('##a', '##t'): 3.107142857142857, ('##t', '##.'): 2.4166666666666665, ('t', '##i'): 0.6041666666666666, ('##i', '##m'): 3.625, ('##m', '##e'): 1.2083333333333333, ('f', '##l'): 4.833333333333333, ('##l', '##i'): 1.8125, ('##i', '##e'): 0.90625, ('##e', '##s'): 2.4166666666666665, ('l', '##i'): 3.625, ('##i', '##k'): 3.625, ('##k', '##e'): 2.4166666666666665, ('a', '##n'): 1.9333333333333333, ('a', '##r'): 3.2222222222222223, ('##r', '##r'): 3.2222222222222223, ('##r', '##o'): 3.2222222222222223, ('##o', '##w'): 4.833333333333333, ('##w', '##;'): 14.5, ('f', '##r'): 3.2222222222222223, ('##r', '##u'): 9.666666666666666, ('##u', '##i'): 3.625, ('##i', '##t'): 0.90625, ('b', '##a'): 2.0714285714285716, ('##n', '##a'): 1.6571428571428573, ('##a', '##.'): 1.380952380952381, ('s', '##h'): 4.833333333333333, ('t', '##o'): 1.6111111111111112, ('##o', '##l'): 2.4166666666666665, ('h', '##i'): 1.8125, ('##h', '##a'): 1.380952380952381, ('k', '##n'): 5.8, ('##n', '##e'): 0.48333333333333334, ('##e', '##w'): 1.2083333333333333, ('h', '##e'): 1.2083333333333333, ('##e', '##d'): 0.8055555555555556, ('##d', '##.'): 3.2222222222222223})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##o', '##w;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}), (('##o', '##w;'), {('t', '##h'): 3.3333333333333335, ('##h', '##e'): 1.6666666666666667, ('o', '##l'): 7.5, ('##l', '##d'): 5.0, ('m', '##a'): 4.285714285714286, ('##a', '##n'): 2.5714285714285716, ('b', '##o'): 5.0, ('##o', '##a'): 1.4285714285714286, ('##a', '##t'): 3.2142857142857144, ('##t', '##.'): 2.5, ('t', '##i'): 0.625, ('##i', '##m'): 3.75, ('##m', '##e'): 1.25, ('f', '##l'): 5.0, ('##l', '##i'): 1.875, ('##i', '##e'): 0.9375, ('##e', '##s'): 2.5, ('l', '##i'): 3.75, ('##i', '##k'): 3.75, ('##k', '##e'): 2.5, ('a', '##n'): 2.0, ('a', '##r'): 3.3333333333333335, ('##r', '##r'): 3.3333333333333335, ('##r', '##o'): 3.3333333333333335, ('##o', '##w;'): 10.0, ('f', '##r'): 3.3333333333333335, ('##r', '##u'): 10.0, ('##u', '##i'): 3.75, ('##i', '##t'): 0.9375, ('b', '##a'): 2.142857142857143, ('##n', '##a'): 1.7142857142857142, ('##a', '##.'): 1.4285714285714286, ('s', '##h'): 5.0, ('t', '##o'): 1.6666666666666667, ('##o', '##l'): 2.5, ('h', '##i'): 1.875, ('##h', '##a'): 1.4285714285714286, ('k', '##n'): 6.0, ('##n', '##e'): 0.5, ('##e', '##w'): 2.5, ('h', '##e'): 1.25, ('##e', '##d'): 0.8333333333333334, ('##d', '##.'): 3.3333333333333335})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##ow;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}), (('##r', '##ow;'), {('t', '##h'): 3.4444444444444446, ('##h', '##e'): 1.7222222222222223, ('o', '##l'): 7.75, ('##l', '##d'): 5.166666666666667, ('m', '##a'): 4.428571428571429, ('##a', '##n'): 2.657142857142857, ('b', '##o'): 7.75, ('##o', '##a'): 2.2142857142857144, ('##a', '##t'): 3.3214285714285716, ('##t', '##.'): 2.5833333333333335, ('t', '##i'): 0.6458333333333334, ('##i', '##m'): 3.875, ('##m', '##e'): 1.2916666666666667, ('f', '##l'): 5.166666666666667, ('##l', '##i'): 1.9375, ('##i', '##e'): 0.96875, ('##e', '##s'): 2.5833333333333335, ('l', '##i'): 3.875, ('##i', '##k'): 3.875, ('##k', '##e'): 2.5833333333333335, ('a', '##n'): 2.066666666666667, ('a', '##r'): 3.4444444444444446, ('##r', '##r'): 3.4444444444444446, ('##r', '##ow;'): 10.333333333333334, ('f', '##r'): 3.4444444444444446, ('##r', '##u'): 10.333333333333334, ('##u', '##i'): 3.875, ('##i', '##t'): 0.96875, ('b', '##a'): 2.2142857142857144, ('##n', '##a'): 1.7714285714285714, ('##a', '##.'): 1.4761904761904763, ('s', '##h'): 5.166666666666667, ('t', '##o'): 2.5833333333333335, ('##o', '##l'): 3.875, ('h', '##i'): 1.9375, ('##h', '##a'): 1.4761904761904763, ('k', '##n'): 6.2, ('##n', '##e'): 0.5166666666666667, ('##e', '##w'): 2.5833333333333335, ('h', '##e'): 1.2916666666666667, ('##e', '##d'): 0.8611111111111112, ('##d', '##.'): 3.4444444444444446})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##row;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}), (('##r', '##row;'), {('t', '##h'): 3.5555555555555554, ('##h', '##e'): 1.7777777777777777, ('o', '##l'): 8.0, ('##l', '##d'): 5.333333333333333, ('m', '##a'): 4.571428571428571, ('##a', '##n'): 2.742857142857143, ('b', '##o'): 8.0, ('##o', '##a'): 2.2857142857142856, ('##a', '##t'): 3.4285714285714284, ('##t', '##.'): 2.6666666666666665, ('t', '##i'): 0.6666666666666666, ('##i', '##m'): 4.0, ('##m', '##e'): 1.3333333333333333, ('f', '##l'): 5.333333333333333, ('##l', '##i'): 2.0, ('##i', '##e'): 1.0, ('##e', '##s'): 2.6666666666666665, ('l', '##i'): 4.0, ('##i', '##k'): 4.0, ('##k', '##e'): 2.6666666666666665, ('a', '##n'): 2.1333333333333333, ('a', '##r'): 5.333333333333333, ('##r', '##row;'): 16.0, ('f', '##r'): 5.333333333333333, ('##r', '##u'): 16.0, ('##u', '##i'): 4.0, ('##i', '##t'): 1.0, ('b', '##a'): 2.2857142857142856, ('##n', '##a'): 1.8285714285714285, ('##a', '##.'): 1.5238095238095237, ('s', '##h'): 5.333333333333333, ('t', '##o'): 2.6666666666666665, ('##o', '##l'): 4.0, ('h', '##i'): 2.0, ('##h', '##a'): 1.5238095238095237, ('k', '##n'): 6.4, ('##n', '##e'): 0.5333333333333333, ('##e', '##w'): 2.6666666666666665, ('h', '##e'): 1.3333333333333333, ('##e', '##d'): 0.8888888888888888, ('##d', '##.'): 3.5555555555555554})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##rrow;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('##r', '##u'), {('t', '##h'): 3.6666666666666665, ('##h', '##e'): 1.8333333333333333, ('o', '##l'): 8.25, ('##l', '##d'): 5.5, ('m', '##a'): 4.714285714285714, ('##a', '##n'): 2.8285714285714287, ('b', '##o'): 8.25, ('##o', '##a'): 2.357142857142857, ('##a', '##t'): 3.5357142857142856, ('##t', '##.'): 2.75, ('t', '##i'): 0.6875, ('##i', '##m'): 4.125, ('##m', '##e'): 1.375, ('f', '##l'): 5.5, ('##l', '##i'): 2.0625, ('##i', '##e'): 1.03125, ('##e', '##s'): 2.75, ('l', '##i'): 4.125, ('##i', '##k'): 4.125, ('##k', '##e'): 2.75, ('a', '##n'): 2.2, ('a', '##rrow;'): 11.0, ('f', '##r'): 11.0, ('##r', '##u'): 33.0, ('##u', '##i'): 4.125, ('##i', '##t'): 1.03125, ('b', '##a'): 2.357142857142857, ('##n', '##a'): 1.8857142857142857, ('##a', '##.'): 1.5714285714285714, ('s', '##h'): 5.5, ('t', '##o'): 2.75, ('##o', '##l'): 4.125, ('h', '##i'): 2.0625, ('##h', '##a'): 1.5714285714285714, ('k', '##n'): 6.6, ('##n', '##e'): 0.55, ('##e', '##w'): 2.75, ('h', '##e'): 1.375, ('##e', '##d'): 0.9166666666666666, ('##d', '##.'): 3.6666666666666665})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##rrow;'), 'fruit': ('f', '##ru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('a', '##rrow;'), {('t', '##h'): 3.7777777777777777, ('##h', '##e'): 1.8888888888888888, ('o', '##l'): 8.5, ('##l', '##d'): 5.666666666666667, ('m', '##a'): 4.857142857142857, ('##a', '##n'): 2.914285714285714, ('b', '##o'): 8.5, ('##o', '##a'): 2.4285714285714284, ('##a', '##t'): 3.642857142857143, ('##t', '##.'): 2.8333333333333335, ('t', '##i'): 0.7083333333333334, ('##i', '##m'): 4.25, ('##m', '##e'): 1.4166666666666667, ('f', '##l'): 5.666666666666667, ('##l', '##i'): 2.125, ('##i', '##e'): 1.0625, ('##e', '##s'): 2.8333333333333335, ('l', '##i'): 4.25, ('##i', '##k'): 4.25, ('##k', '##e'): 2.8333333333333335, ('a', '##n'): 2.2666666666666666, ('a', '##rrow;'): 11.333333333333334, ('f', '##ru'): 11.333333333333334, ('##ru', '##i'): 4.25, ('##i', '##t'): 1.0625, ('b', '##a'): 2.4285714285714284, ('##n', '##a'): 1.9428571428571428, ('##a', '##.'): 1.619047619047619, ('s', '##h'): 5.666666666666667, ('t', '##o'): 2.8333333333333335, ('##o', '##l'): 4.25, ('h', '##i'): 2.125, ('##h', '##a'): 1.619047619047619, ('k', '##n'): 6.8, ('##n', '##e'): 0.5666666666666667, ('##e', '##w'): 2.8333333333333335, ('h', '##e'): 1.4166666666666667, ('##e', '##d'): 0.9444444444444444, ('##d', '##.'): 3.7777777777777777})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('f', '##ru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('f', '##ru'), {('t', '##h'): 3.888888888888889, ('##h', '##e'): 1.9444444444444444, ('o', '##l'): 8.75, ('##l', '##d'): 5.833333333333333, ('m', '##a'): 5.0, ('##a', '##n'): 3.0, ('b', '##o'): 8.75, ('##o', '##a'): 2.5, ('##a', '##t'): 3.75, ('##t', '##.'): 2.9166666666666665, ('t', '##i'): 0.7291666666666666, ('##i', '##m'): 4.375, ('##m', '##e'): 1.4583333333333333, ('f', '##l'): 5.833333333333333, ('##l', '##i'): 2.1875, ('##i', '##e'): 1.09375, ('##e', '##s'): 2.9166666666666665, ('l', '##i'): 4.375, ('##i', '##k'): 4.375, ('##k', '##e'): 2.9166666666666665, ('a', '##n'): 3.5, ('f', '##ru'): 11.666666666666666, ('##ru', '##i'): 4.375, ('##i', '##t'): 1.09375, ('b', '##a'): 2.5, ('##n', '##a'): 2.0, ('##a', '##.'): 1.6666666666666667, ('s', '##h'): 5.833333333333333, ('t', '##o'): 2.9166666666666665, ('##o', '##l'): 4.375, ('h', '##i'): 2.1875, ('##h', '##a'): 1.6666666666666667, ('k', '##n'): 7.0, ('##n', '##e'): 0.5833333333333334, ('##e', '##w'): 2.9166666666666665, ('h', '##e'): 1.4583333333333333, ('##e', '##d'): 0.9722222222222222, ('##d', '##.'): 3.888888888888889})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', 'fru', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('b', '##o'), {('t', '##h'): 4.0, ('##h', '##e'): 2.0, ('o', '##l'): 9.0, ('##l', '##d'): 6.0, ('m', '##a'): 5.142857142857143, ('##a', '##n'): 3.085714285714286, ('b', '##o'): 9.0, ('##o', '##a'): 2.5714285714285716, ('##a', '##t'): 3.857142857142857, ('##t', '##.'): 3.0, ('t', '##i'): 0.75, ('##i', '##m'): 4.5, ('##m', '##e'): 1.5, ('f', '##l'): 9.0, ('##l', '##i'): 2.25, ('##i', '##e'): 1.125, ('##e', '##s'): 3.0, ('l', '##i'): 4.5, ('##i', '##k'): 4.5, ('##k', '##e'): 3.0, ('a', '##n'): 3.6, ('fru', '##i'): 4.5, ('##i', '##t'): 1.125, ('b', '##a'): 2.5714285714285716, ('##n', '##a'): 2.057142857142857, ('##a', '##.'): 1.7142857142857142, ('s', '##h'): 6.0, ('t', '##o'): 3.0, ('##o', '##l'): 4.5, ('h', '##i'): 2.25, ('##h', '##a'): 1.7142857142857142, ('k', '##n'): 7.2, ('##n', '##e'): 0.6, ('##e', '##w'): 3.0, ('h', '##e'): 1.5, ('##e', '##d'): 1.0, ('##d', '##.'): 4.0})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', 'fru', 'bo', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('##o', '##l'), {('t', '##h'): 4.111111111111111, ('##h', '##e'): 2.0555555555555554, ('o', '##l'): 9.25, ('##l', '##d'): 6.166666666666667, ('m', '##a'): 5.285714285714286, ('##a', '##n'): 3.1714285714285713, ('bo', '##a'): 5.285714285714286, ('##a', '##t'): 3.9642857142857144, ('##t', '##.'): 3.0833333333333335, ('t', '##i'): 0.7708333333333334, ('##i', '##m'): 4.625, ('##m', '##e'): 1.5416666666666667, ('f', '##l'): 9.25, ('##l', '##i'): 2.3125, ('##i', '##e'): 1.15625, ('##e', '##s'): 3.0833333333333335, ('l', '##i'): 4.625, ('##i', '##k'): 4.625, ('##k', '##e'): 3.0833333333333335, ('a', '##n'): 3.7, ('fru', '##i'): 4.625, ('##i', '##t'): 1.15625, ('b', '##a'): 5.285714285714286, ('##n', '##a'): 2.1142857142857143, ('##a', '##.'): 1.7619047619047619, ('s', '##h'): 6.166666666666667, ('t', '##o'): 6.166666666666667, ('##o', '##l'): 9.25, ('h', '##i'): 2.3125, ('##h', '##a'): 1.7619047619047619, ('k', '##n'): 7.4, ('##n', '##e'): 0.6166666666666667, ('##e', '##w'): 3.0833333333333335, ('h', '##e'): 1.5416666666666667, ('##e', '##d'): 1.0277777777777777, ('##d', '##.'): 4.111111111111111})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##ol', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('##ol', '##d'), {('t', '##h'): 4.222222222222222, ('##h', '##e'): 2.111111111111111, ('o', '##l'): 12.666666666666666, ('##l', '##d'): 4.222222222222222, ('m', '##a'): 5.428571428571429, ('##a', '##n'): 3.257142857142857, ('bo', '##a'): 5.428571428571429, ('##a', '##t'): 4.071428571428571, ('##t', '##.'): 3.1666666666666665, ('t', '##i'): 0.7916666666666666, ('##i', '##m'): 4.75, ('##m', '##e'): 1.5833333333333333, ('f', '##l'): 12.666666666666666, ('##l', '##i'): 3.1666666666666665, ('##i', '##e'): 1.1875, ('##e', '##s'): 3.1666666666666665, ('l', '##i'): 4.75, ('##i', '##k'): 4.75, ('##k', '##e'): 3.1666666666666665, ('a', '##n'): 3.8, ('fru', '##i'): 4.75, ('##i', '##t'): 1.1875, ('b', '##a'): 5.428571428571429, ('##n', '##a'): 2.1714285714285713, ('##a', '##.'): 1.8095238095238095, ('s', '##h'): 6.333333333333333, ('t', '##ol'): 6.333333333333333, ('##ol', '##d'): 12.666666666666666, ('h', '##i'): 2.375, ('##h', '##a'): 1.8095238095238095, ('k', '##n'): 7.6, ('##n', '##e'): 0.6333333333333333, ('##e', '##w'): 3.1666666666666665, ('h', '##e'): 1.5833333333333333, ('##e', '##d'): 1.0555555555555556, ('##d', '##.'): 4.222222222222222})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('f', '##l'), {('t', '##h'): 4.333333333333333, ('##h', '##e'): 2.1666666666666665, ('o', '##l'): 13.0, ('##l', '##d'): 6.5, ('m', '##a'): 5.571428571428571, ('##a', '##n'): 3.342857142857143, ('bo', '##a'): 5.571428571428571, ('##a', '##t'): 4.178571428571429, ('##t', '##.'): 3.25, ('t', '##i'): 0.8125, ('##i', '##m'): 4.875, ('##m', '##e'): 1.625, ('f', '##l'): 13.0, ('##l', '##i'): 3.25, ('##i', '##e'): 1.21875, ('##e', '##s'): 3.25, ('l', '##i'): 4.875, ('##i', '##k'): 4.875, ('##k', '##e'): 3.25, ('a', '##n'): 3.9, ('fru', '##i'): 4.875, ('##i', '##t'): 1.21875, ('b', '##a'): 5.571428571428571, ('##n', '##a'): 2.2285714285714286, ('##a', '##.'): 1.8571428571428572, ('s', '##h'): 6.5, ('t', '##old'): 6.5, ('h', '##i'): 2.4375, ('##h', '##a'): 1.8571428571428572, ('k', '##n'): 7.8, ('##n', '##e'): 0.65, ('##e', '##w'): 3.25, ('h', '##e'): 1.625, ('##e', '##d'): 1.625, ('##d', '##.'): 6.5})), (({'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('o', '##l'), {('t', '##h'): 4.444444444444445, ('##h', '##e'): 2.2222222222222223, ('o', '##l'): 40.0, ('##l', '##d'): 20.0, ('m', '##a'): 5.714285714285714, ('##a', '##n'): 3.4285714285714284, ('bo', '##a'): 5.714285714285714, ('##a', '##t'): 4.285714285714286, ('##t', '##.'): 3.3333333333333335, ('t', '##i'): 0.8333333333333334, ('##i', '##m'): 5.0, ('##m', '##e'): 1.6666666666666667, ('fl', '##i'): 5.0, ('##i', '##e'): 1.25, ('##e', '##s'): 3.3333333333333335, ('l', '##i'): 5.0, ('##i', '##k'): 5.0, ('##k', '##e'): 3.3333333333333335, ('a', '##n'): 4.0, ('fru', '##i'): 5.0, ('##i', '##t'): 1.25, ('b', '##a'): 5.714285714285714, ('##n', '##a'): 2.2857142857142856, ('##a', '##.'): 1.9047619047619047, ('s', '##h'): 6.666666666666667, ('t', '##old'): 6.666666666666667, ('h', '##i'): 2.5, ('##h', '##a'): 1.9047619047619047, ('k', '##n'): 8.0, ('##n', '##e'): 0.6666666666666666, ('##e', '##w'): 3.3333333333333335, ('h', '##e'): 1.6666666666666667, ('##e', '##d'): 1.6666666666666667, ('##d', '##.'): 6.666666666666667})), (({'the': ('t', '##h', '##e'), 'old': ('ol', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('ol', '##d'), {('t', '##h'): 4.555555555555555, ('##h', '##e'): 2.2777777777777777, ('ol', '##d'): 20.5, ('m', '##a'): 5.857142857142857, ('##a', '##n'): 3.5142857142857142, ('bo', '##a'): 5.857142857142857, ('##a', '##t'): 4.392857142857143, ('##t', '##.'): 3.4166666666666665, ('t', '##i'): 0.8541666666666666, ('##i', '##m'): 5.125, ('##m', '##e'): 1.7083333333333333, ('fl', '##i'): 5.125, ('##i', '##e'): 1.28125, ('##e', '##s'): 3.4166666666666665, ('l', '##i'): 5.125, ('##i', '##k'): 5.125, ('##k', '##e'): 3.4166666666666665, ('a', '##n'): 4.1, ('fru', '##i'): 5.125, ('##i', '##t'): 1.28125, ('b', '##a'): 5.857142857142857, ('##n', '##a'): 2.342857142857143, ('##a', '##.'): 1.9523809523809523, ('s', '##h'): 6.833333333333333, ('t', '##old'): 6.833333333333333, ('h', '##i'): 2.5625, ('##h', '##a'): 1.9523809523809523, ('k', '##n'): 8.2, ('##n', '##e'): 0.6833333333333333, ('##e', '##w'): 3.4166666666666665, ('h', '##e'): 1.7083333333333333, ('##e', '##d'): 1.7083333333333333, ('##d', '##.'): 6.833333333333333})), (({'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('##d', '##.'), {('t', '##h'): 4.666666666666667, ('##h', '##e'): 2.3333333333333335, ('m', '##a'): 6.0, ('##a', '##n'): 3.6, ('bo', '##a'): 6.0, ('##a', '##t'): 4.5, ('##t', '##.'): 3.5, ('t', '##i'): 0.875, ('##i', '##m'): 5.25, ('##m', '##e'): 1.75, ('fl', '##i'): 5.25, ('##i', '##e'): 1.3125, ('##e', '##s'): 3.5, ('l', '##i'): 5.25, ('##i', '##k'): 5.25, ('##k', '##e'): 3.5, ('a', '##n'): 4.2, ('fru', '##i'): 5.25, ('##i', '##t'): 1.3125, ('b', '##a'): 6.0, ('##n', '##a'): 2.4, ('##a', '##.'): 2.0, ('s', '##h'): 7.0, ('t', '##old'): 7.0, ('h', '##i'): 2.625, ('##h', '##a'): 2.0, ('k', '##n'): 8.4, ('##n', '##e'): 0.7, ('##e', '##w'): 3.5, ('h', '##e'): 1.75, ('##e', '##d'): 3.5, ('##d', '##.'): 14.0})), (({'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d.')}, {'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##d.', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}), (('k', '##n'), {('t', '##h'): 4.777777777777778, ('##h', '##e'): 2.388888888888889, ('m', '##a'): 6.142857142857143, ('##a', '##n'): 3.6857142857142855, ('bo', '##a'): 6.142857142857143, ('##a', '##t'): 4.607142857142857, ('##t', '##.'): 5.375, ('t', '##i'): 0.8958333333333334, ('##i', '##m'): 5.375, ('##m', '##e'): 1.7916666666666667, ('fl', '##i'): 5.375, ('##i', '##e'): 1.34375, ('##e', '##s'): 3.5833333333333335, ('l', '##i'): 5.375, ('##i', '##k'): 5.375, ('##k', '##e'): 3.5833333333333335, ('a', '##n'): 4.3, ('fru', '##i'): 5.375, ('##i', '##t'): 1.34375, ('b', '##a'): 6.142857142857143, ('##n', '##a'): 2.4571428571428573, ('##a', '##.'): 3.0714285714285716, ('s', '##h'): 7.166666666666667, ('t', '##old'): 7.166666666666667, ('h', '##i'): 2.6875, ('##h', '##a'): 2.0476190476190474, ('k', '##n'): 8.6, ('##n', '##e'): 0.7166666666666667, ('##e', '##w'): 3.5833333333333335, ('h', '##e'): 1.7916666666666667, ('##e', '##d.'): 3.5833333333333335})), (({'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('kn', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d.')}, {'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##d.', '##.', '##rrow;', 's', 'kn', 'o', '##h', '##s', 'f'}), (('s', '##h'), {('t', '##h'): 4.888888888888889, ('##h', '##e'): 2.4444444444444446, ('m', '##a'): 6.285714285714286, ('##a', '##n'): 4.714285714285714, ('bo', '##a'): 6.285714285714286, ('##a', '##t'): 4.714285714285714, ('##t', '##.'): 5.5, ('t', '##i'): 0.9166666666666666, ('##i', '##m'): 5.5, ('##m', '##e'): 1.8333333333333333, ('fl', '##i'): 5.5, ('##i', '##e'): 1.375, ('##e', '##s'): 3.6666666666666665, ('l', '##i'): 5.5, ('##i', '##k'): 5.5, ('##k', '##e'): 3.6666666666666665, ('a', '##n'): 5.5, ('fru', '##i'): 5.5, ('##i', '##t'): 1.375, ('b', '##a'): 6.285714285714286, ('##n', '##a'): 3.142857142857143, ('##a', '##.'): 3.142857142857143, ('s', '##h'): 7.333333333333333, ('t', '##old'): 7.333333333333333, ('h', '##i'): 2.75, ('##h', '##a'): 2.0952380952380953, ('kn', '##e'): 3.6666666666666665, ('##e', '##w'): 3.6666666666666665, ('h', '##e'): 1.8333333333333333, ('##e', '##d.'): 3.6666666666666665}))])

    word_freqs, test_cases = test_cases[0], test_cases[1]
    tokenizer = WordPieceTokenizer(train, 45, do_print=False, do_tqdm=False)

    overall_pass = True
    for i in range(len(test_cases)):
        res = tokenizer.find_best_pair(test_cases[i][0][0], word_freqs, test_cases[i][0][1])
        if len(res) != 2:
            pass1, msg1 = 'INCORRECT', 'Did not return 2 items.'
        else:
            best_pair, scores = res
            bp = best_pair == test_cases[i][1][0]
            sc, scc = check_dictionary(scores, test_cases[i][1][1])
            if bp and sc == 'CORRECT':
                pass1, msg1 = 'CORRECT', ''
            else:
                overall_pass = False
                pass1 = 'INCORRECT'
                msg1 = ''
                if not bp: msg1 += 'best_pair is incorrect. '
                if sc == 'INCORRECT': msg1 += 'scores is incorrect (' + scc + ')'
        print('\tCase ' + (' ' if i < 10 else '') + str(i) + ':\t' + pass1 + '\t' + msg1 + '\tword_tokenizations: ' + str(test_cases[i][0][0]) + '\tword_freqs: ' + str(word_freqs) + '\tvocab: ' + str(test_cases[i][0][1]) +
              '\tYour best_pair: ' + str(best_pair) + '\tCorrect best_pair: ' + str(test_cases[i][1][0]) + '\tYour scores: ' + str(scores) + '\tCorrect scores: ' + str(test_cases[i][1][1]))
    if overall_pass: print('\n  Passed!')
    else: print('  Failed.')

def sanityCheckMergeBestPair(train):
    print('\n\n--- TEST: merge_best_pair(self, best_pair, vocab, word_tokenizations, tokens2word) ---') # max_len does not matter for this test
    test_cases = [((('##w', '##;'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##o', '##w', '##;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##o', '##w;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}})), ((('##o', '##w;'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##o', '##w;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##ow;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}})), ((('##r', '##ow;'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##r', '##ow;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##row;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}})), ((('##r', '##row;'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##r', '##row;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##rrow;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}})), ((('##r', '##u'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##rrow;'), 'fruit': ('f', '##r', '##u', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##rrow;'), 'fruit': ('f', '##ru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}})), ((('a', '##rrow;'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('a', '##rrow;'), 'fruit': ('f', '##ru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('f', '##ru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}})), ((('f', '##ru'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('f', '##ru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', 'fru', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}})), ((('b', '##o'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', 'fru', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('b', '##o', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', 'fru', 'bo', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}})), ((('##o', '##l'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', 'fru', 'bo', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##o', '##l', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##ol', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}})), ((('##ol', '##d'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##ol', '##d'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}})), ((('f', '##l'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('f', '##l', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}})), ((('o', '##l'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('o', '##l', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}}), ({'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('ol', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}})), ((('ol', '##d'), {'<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('ol', '##d'), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}}), ({'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}, 'old': {'old'}})), ((('##d', '##.'), {'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d', '##.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}, 'old': {'old'}}), ({'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##d.', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}, 'old': {'old'}, '##d.': {'lied.'}})), ((('k', '##n'), {'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##d.', '##.', '##rrow;', 's', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('k', '##n', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}, 'old': {'old'}, '##d.': {'lied.'}}), ({'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##d.', '##.', '##rrow;', 's', 'kn', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('kn', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}, 'old': {'old'}, '##d.': {'lied.'}, 'kn': {'knew'}})), ((('s', '##h'), {'old', '<UNK>', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##d.', '##.', '##rrow;', 's', 'kn', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('s', '##h', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('kn', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}, 'old': {'old'}, '##d.': {'lied.'}, 'kn': {'knew'}}), ({'old', '<UNK>', 'sh', 'm', 'h', '##t', 'a', '##a', '##k', '##ow;', '##ol', 'fru', 'bo', '##i', '##m', '##old', '##row;', 't', '##d', 'l', '##r', '##o', 'fl', 'ol', 'arrow;', '##u', 'k', '##n', '##w;', 'b', '##e', '##ru', '##w', '##l', ' ', '##;', '##d.', '##.', '##rrow;', 's', 'kn', 'o', '##h', '##s', 'f'}, {'the': ('t', '##h', '##e'), 'old': ('old',), 'man': ('m', '##a', '##n'), 'boat.': ('bo', '##a', '##t', '##.'), 'time': ('t', '##i', '##m', '##e'), 'flies': ('fl', '##i', '##e', '##s'), 'like': ('l', '##i', '##k', '##e'), 'an': ('a', '##n'), 'arrow;': ('arrow;',), 'fruit': ('fru', '##i', '##t'), 'a': ('a',), 'banana.': ('b', '##a', '##n', '##a', '##n', '##a', '##.'), 'she': ('sh', '##e'), 'told': ('t', '##old'), 'him': ('h', '##i', '##m'), 'that': ('t', '##h', '##a', '##t'), 'knew': ('kn', '##e', '##w'), 'he': ('h', '##e'), 'lied.': ('l', '##i', '##e', '##d.')}, {'t': {'time', 'the', 'that', 'told'}, '##h': {'she', 'the', 'that'}, '##e': {'knew', 'the', 'like', 'time', 'she', 'he', 'flies', 'lied.'}, 'o': {'old'}, '##l': {'old', 'flies', 'told'}, '##d': {'old', 'told', 'lied.'}, 'm': {'man'}, '##a': {'boat.', 'banana.', 'that', 'man'}, '##n': {'an', 'knew', 'banana.', 'man'}, 'b': {'banana.', 'boat.'}, '##o': {'arrow;', 'boat.', 'told'}, '##t': {'boat.', 'that', 'fruit'}, '##.': {'banana.', 'boat.', 'lied.'}, '##i': {'him', 'like', 'time', 'fruit', 'flies', 'lied.'}, '##m': {'time', 'him'}, 'f': {'flies', 'fruit'}, '##s': {'flies'}, 'l': {'like', 'lied.'}, '##k': {'like'}, 'a': {'an', 'a', 'arrow;'}, '##r': {'arrow;', 'fruit'}, '##w': {'knew', 'arrow;'}, '##;': {'arrow;'}, '##u': {'fruit'}, 's': {'she'}, 'h': {'he', 'him'}, 'k': {'knew'}, '##w;': {'arrow;'}, '##ow;': {'arrow;'}, '##row;': {'arrow;'}, '##rrow;': {'arrow;'}, '##ru': {'fruit'}, 'arrow;': {'arrow;'}, 'fru': {'fruit'}, 'bo': {'boat.'}, '##ol': {'told'}, '##old': {'told'}, 'fl': {'flies'}, 'ol': {'old'}, 'old': {'old'}, '##d.': {'lied.'}, 'kn': {'knew'}, 'sh': {'she'}}))]

    tokenizer = WordPieceTokenizer(train, 45, do_print=False, do_tqdm=False)

    overall_pass = True
    for i in range(len(test_cases)):
        from copy import deepcopy
        vocab, word_tokenizations, tokens2word = deepcopy(test_cases[i][0][1]), deepcopy(test_cases[i][0][2]), deepcopy(test_cases[i][0][3])
        vocabo, word_tokenizationso, tokens2wordo = deepcopy(vocab), deepcopy(word_tokenizations), deepcopy(tokens2word)
        tokenizer.merge_best_pair(test_cases[i][0][0], vocab, word_tokenizations, tokens2word)

        pass1, msg0 = check_set(vocab, test_cases[i][1][0])
        pass2, msg2 = check_dictionary(word_tokenizations, test_cases[i][1][1])
        pass3, msg3 = check_dictionary(tokens2word, test_cases[i][1][2])
        if pass1 == 'CORRECT' and pass2 == 'CORRECT' and pass3 == 'CORRECT':
            pass1, msg1 = 'CORRECT', ''
        else:
            overall_pass = False
            pass1 = 'INCORRECT'
            msg1 = ''
            if pass1 != 'CORRECT': msg1 += 'vocab is incorrect. (' + msg0 + ') '
            if pass2 != 'CORRECT': msg1 += 'word_tokenizations is incorrect. (' + msg2 + ') '
            if pass3 != 'CORRECT': msg1 += 'tokens2word is incorrect. (' + msg3 + ') '
        print('\tCase ' + (' ' if i < 10 else '') + str(i) + ':\t' + pass1 + '\t' + msg1 + '\tbest_pair: ' + str(test_cases[i][0][0]) + '\tvocab: ' + str(vocabo) + '\tword_tokenizations: ' + str(word_tokenizationso) + '\ttokens2word: ' + str(tokens2wordo) +
              '\tYour vocab: ' + str(vocab) + '\tCorrect vocab: ' + str(test_cases[i][1][0]) + '\tYour word_tokenizations: ' + str(word_tokenizations) + '\tCorrect word_tokenizations: ' + str(test_cases[i][1][1])+
              '\tYour tokens2word: ' + str(tokens2word) + '\tCorrect tokens2word: ' + str(test_cases[i][1][2]))
    if overall_pass: print('\n  Passed!')
    else: print('  Failed.')


def sanityCheckTokenize(train, test):

    print('\n\n--- TEST: tokenize(self, sentence) ---')

    tokenizer = WordPieceTokenizer(train, 45, do_print=False, do_tqdm=False)
    tokenizer.vocab = [' ', '##.', '##;', '##a', '##d', '##d.', '##e', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##ol', '##old', '##ow;', '##r', '##row;', '##rrow;', '##ru', '##s', '##t', '##u', '##w', '##w;', '<UNK>', 'a', 'arrow;', 'b', 'bo', 'f', 'fl', 'fru', 'h', 'k', 'kn', 'l', 'm', 'o', 'ol', 'old', 's', 'sh', 't']
    correct_ans = [['sh', '##e', ' ', '<UNK>', ' ', 't', '##h', '##e', ' ', 'bo', '##o', '##k', ' ', '<UNK>'], ['t', '##i', '##m', '##e', ' ', '<UNK>', ' ', 'f', '##o', '##r', ' ', '<UNK>', ' ', 'o', '##n', '##e', '##.']]

    overall_pass = True
    for i in range(len(test)):
        toks = tokenizer.tokenize(test[i])
        if toks == correct_ans[i]: pass1, msg1 = 'CORRECT', ''
        else:
            pass1, msg1 = 'INCORRECT', 'Tokenization is incorrect.'
            overall_pass = False
        print('\tCase ' + (' ' if i < 10 else '') + str(i) + ':\t' + pass1 + '\t' + msg1 + '\tsentence: ' + str(test[i]) +
              '\tYour tokenization: ' + str(toks) + '\tCorrect tokenization: ' + str(correct_ans[i]))

    if overall_pass: print('\n  Passed!')
    else: print('  Failed.')

def sanityCheckDetokenize(train):

    print('\n\n--- TEST: detokenize(self, tokens) ---')

    tokenizer = WordPieceTokenizer(train, 45, do_print=False, do_tqdm=False)
    tokenizer.vocab = [' ', '##.', '##;', '##a', '##d', '##d.', '##e', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##ol', '##old', '##ow;', '##r', '##row;', '##rrow;', '##ru', '##s', '##t', '##u', '##w', '##w;', '<UNK>', 'a', 'arrow;', 'b', 'bo', 'f', 'fl', 'fru', 'h', 'k', 'kn', 'l', 'm', 'o', 'ol', 'old', 's', 'sh', 't']
    inputs = [['sh', '##e', ' ', '<UNK>', ' ', 't', '##h', '##e', ' ', 'bo', '##o', '##k', ' ', '<UNK>'], ['t', '##i', '##m', '##e', ' ', '<UNK>', ' ', 'f', '##o', '##r', ' ', '<UNK>', ' ', 'o', '##n', '##e', '##.']]
    correct_ans = ['she <UNK> the book <UNK>', 'time <UNK> for <UNK> one.']

    overall_pass = True
    for i in range(len(inputs)):
        sent = tokenizer.detokenize(inputs[i])
        if sent == correct_ans[i]: pass1, msg1 = 'CORRECT', ''
        else:
            pass1, msg1 = 'INCORRECT', 'Sentence is incorrect.'
            overall_pass = False
        print('\tCase ' + (' ' if i < 10 else '') + str(i) + ':\t' + pass1 + '\t' + msg1 + '\ttokens: ' + str(inputs[i]) +
              '\tYour sentence: ' + str(sent) + '\tCorrect sentence: ' + str(correct_ans[i]))

    if overall_pass: print('\n  Passed!')
    else: print('  Failed.')


def sanityCheckTokenizer():
    sample_train_corpus = ["the old man the boat.",
                   "time flies like an arrow; fruit flies like a banana.",
                   "she told him that she knew that he lied."]
    sample_test_corpus = ["she read the book 1984.",
                          "time waits for no one."]

    print("Sample train corpus:", sample_train_corpus)
    print("Sample test corpus:", sample_test_corpus)
    sanityCheckInitialize(sample_train_corpus)
    sanityCheckFindBestPair(sample_train_corpus)
    sanityCheckMergeBestPair(sample_train_corpus)
    sanityCheckTokenize(sample_train_corpus, sample_test_corpus)
    sanityCheckDetokenize(sample_train_corpus)


In [ ]:
if __name__=='__main__':
    sanityCheckTokenizer()

## Run tokenizer

In [ ]:

if __name__=='__main__':
    tokenizer = WordPieceTokenizer(dataset.train, 700, do_tqdm = True, do_print=False)
    tokenizer.train()
    print("Vocab:", tokenizer.vocab)

Now we can look at some tokenizations on the test set.

In [ ]:

if __name__=='__main__':
    for sent in random.sample(dataset.test, 10):
        print('Sentence:', sent)
        toks = tokenizer.tokenize(sent)
        print('Tokenization:', toks)
        print('Detokenization:', tokenizer.detokenize(toks))
        print()

# Part 2: Language Models

Here, we will train some <b>n-gram language models</b> on WikiText-2, a corpus of high-quality Wikipedia articles. The dataset was originally introduced in the following paper: https://arxiv.org/pdf/1609.07843v1.pdf. A raw version of the data can easily be viewed here: https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2.

## Download & preprocess the data

To make your models more robust, it is necessary to perform some basic preprocessing on the corpora. 

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp;We are interested in modeling individual sentences, rather than longer chunks of text such as paragraphs or documents. The WikiTest dataset provides paragraphs; thus, we provide a simple method to identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`). These markers will allow your models to generate sentences that have realistic beginnings and endings.

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;In order to deal with unknown words in the test corpora, all words that do not appear in the vocabulary must be replaced with a special token for unknown words (`<UNK>`) before estimating your models. The WikiText dataset has already done this, and you can read about the method in the paper above. When unknown words are encountered in the test corpus, they should be treated as that special token instead.

After the preprocessing, it is assumed that all words in the test set appear in the training set, as this code has already replaced the unseen tokens with `<UNK>`.

In [ ]:

START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [ ]:
import os
import random
import sys
from urllib.request import urlretrieve

def preprocess(data, vocab=None):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = set()
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    splits = ['train', 'valid']
    datasets = []
    path = './{}.txt'
    url = 'https://raw.githubusercontent.com/pytorch/examples/main/word_language_model/data/wikitext-2/{}.txt'
    for split in splits:
        if os.path.exists(path.format(split)):
            print(f"{split} dataset already downloaded")
        else:
            filename = f'{split}.txt'
            urlretrieve(url.format(split), filename)
        datasets.append(open(f'{split}.txt').read().split('\n'))
    train_dataset, vocab = preprocess(datasets[0])
    test_dataset, _ = preprocess(datasets[1], vocab)
    return train_dataset, test_dataset

if __name__ == '__main__':
    train_dataset, test_dataset = getDataset()

Run the next cell to see 10 random sentences of the training data.

In [ ]:
if __name__ == '__main__':
    for x in random.sample(train_dataset, 10):
        print (x)

## The LanguageModel Class

We will implement 4 types of language models: a <b>unigram</b> model, a <b>smoothed unigram</b> model, a <b>bigram</b> model, and a <b>smoothed bigram</b> model. Each of the models is worth 25 points and extends the following base class. <b>You do not need to implement anything in this class</b>; you will instead implement each of the following methods in the relevant subclass:

* <b>`__init__(self, trainCorpus)`</b>: Train the language model on `trainCorpus`. This will involve calculating relative frequency estimates according to the type of model you're implementing.

* <b>`generateSentence(self)`</b>:  Return a sentence that is generated by the language model. It should be a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>, where each <TT>w<sup>(i)</sup></TT> is a word in your vocabulary (including <TT>&lt;UNK&gt;</TT> but exlcuding <TT>&lt;s&gt;</TT> and <TT>&lt;&sol;s&gt;</TT>). You may assume that <TT>&lt;s&gt;</TT> starts each sentence (with probability $1$). The following words <TT>w<sup>(1)</sup></TT>, ... , <TT>w<sup>(n)</sup></TT>, <TT>&lt;&sol;s&gt;</TT> are generated according to your language model's distribution. Note that the number of words <TT>n</TT> is not fixed; instead, you should stop the sentence as soon as you generate the stop token <TT>&lt;&sol;s&gt;</TT>.

* <b>`getSentenceLogProbability(self, sentence)`</b>: Return the <em> logarithm of the probability</em> of <TT>sentence</TT>, which is again a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>. You should use the natural logarithm $-$ that is, the base-<em>e</em> logarithm. See the note below about performing your calculations in log space.

* <b>`getCorpusPerplexity(self, testCorpus)`</b>:  We need to compute the perplexity (normalized inverse log probability) of `testCorpus` according to your model. For a corpus $W$ with $N$ words and a bigram model, Jurafsky and Martin tells you to compute perplexity as follows:

$$Perplexity(W) = \Big [ \prod_{i=1}^N \frac{1}{P(w^{(i)}|w^{(i-1)})} \Big ]^{1/N}$$

In order to avoid underflow, all calculations are done in log-space. That is, instead of multiplying probabilities, we add the logarithms of the probabilities and exponentiate the result:

$$\prod_{i=1}^N P(w^{(i)}|w^{(i-1)}) = \exp\Big (\sum_{i=1}^N \log P(w^{(i)}|w^{(i-1)}) \Big ) $$
